# Aggregation of Power Systems Optimization Models - Part I

Consider the following model:

![alt text](model.png "Model")

The goal is to find $r$ such that $|r|<<|h|$ but keeping results as close as posible to thoseobtained with $h$.

In the following cells an example is shown using a testbed using **Pyomo** to run aggregated and complete versions of this model.


In [22]:
import os
import pyomo.environ as pyo
import testbed
from importlib import reload


In [29]:
# We can export the variables, first create a folder and then export the results
slvr = 'gurobi'     # change according to the solver installed in your machine
reload(testbed)
results_complete = testbed.run_complete_case(folder='data', case='complete_single_node.xlsx', solver=slvr)
# exporting the duals
df_duals_complete = testbed.extract_duals(model=results_complete, folder='complete_single_node')
df_complete = testbed.export_complete_solution(results_complete, '.')

In [30]:
results = testbed.basis_execution(folder=os.path.join('data', 'aggregated_single'), # location of the aggregated model
                                  solver=slvr, file='config.xlsx')                  # configuration file with aggregation information
df_results = testbed.export_aggregated_solution(results, 'aggregated_single')
models = testbed.basis_execution(folder='data/aggregated_single')
df_agg = testbed.export_aggregated_solution(models, 'aggregated_single')
df_comparison = testbed.export_model_comparison(df_complete, df_agg)

In [32]:
df_agg

,basis,of_value,thermal,renewable,nsp,weight
0,bs1,9174000.0,278000.0,834000.0,0.0,5560
1,bs2,13950000.0,450000.0,1050000.0,0.0,3000


In [33]:
df_comparison

,result,complete,aggregated,delta
0,of_value,6.387533e+07,23124000.0,4.075133e+07
1,thermal,2.505880e+06,728000.0,1.777880e+06
2,renewable,1.244741e+06,1884000.0,-6.392594e+05
3,nsp,0.000000e+00,0.0,0.000000e+00
